# XGBoost

XGBoost (extreme gradient boosting) 极致梯度提升，是一种基于GBDT的算法或者说工程实现。

XGBoost的基本思想和GBDT相同，但是做了一些优化，比如二阶导数使函数更精准；正则化避免树过拟合；Block存储可以并行计算等。

## 1. XGBoost目标函数推导

XGBoost的目标函数由损失函数和正则化两部分组成。

已知训练数据集$T=\{(x_1,y_1),(x_2,y_2),...,(x_n,y_n)\}$，损失函数$l(y_i,\hat(y)_i)$，正则化项$\Omega(f_t)$，则整体目标函数可记为：
$$L(\phi)=\sum_i{l(y_i,\hat{y}_i)} + \sum_j{\Omega(f_j)}$$

其中：
- $L(\phi)$是线性空间上的表达
- i是第i个样本，j是第j颗数。
- $\hat{y}_i$是第i个样本$x_i$的预测值
$$\hat{y}_i = \sum_{j=1}^K{f_j(x_i)}$$

两个累加的变量是不同的：

- 一个是i,i这边代表的是样本的数量，也就是对每个样本我们都会做一个损失的计算，这个损失是第t个树的预测值与真实值之间的差值计算（具体如何度量损失视情况而定）
- 另一个是累加变量是j, 代表的是树的数量，也就是我们每棵树的复杂度进行累加。

**需要注意的是我们这里具体的损失函数是没有给出定义，所以它可以是树模型，也可以是线性模型。**



### 1.1 简化损失函数

XGBoost是向前迭代算法，在第t次迭代时，前t-1次迭代产生的树变量可以看成常量：

$$Obj^{(t)}=\sum_{i=1}^n{l(y_i,\hat{y}_i^{(t)})} + \sum_{j=1}^t{\Omega(f_j)}$$
$$=\sum_{i=1}^n{l(y_i,\hat{y}^{(t-1)}+f_t(x_i))} +\sum_{j=1}^t{\Omega(f_j)}$$
$$=\sum_{i=1}^n{l(y_i,\hat{y}_i^{t-1}+f_t(x_i))} + \Omega(f_t)+constant$$

### 1.2 泰勒公式对目标函数近似展开

对$l(y_i,\hat{y}_i^{t-1}+f_t(x_i))$在 $\hat{y}_i^{(t-1)}$泰勒二阶展开：

$$l(y_i,\hat{y}_i^{t-1}+f_t(x_i)) = l(y_i,\hat{y}_i^{(t-1)})+g_if_t(x_i)+\frac{1}{2}h_if_t^2(x_i)$$

故原损失函数为：

$$Obj^{(t)}=\sum_{i=1}^n{[l(y_i,\hat{y}_i^{(t-1)})+g_if_t(x_i)+\frac{1}{2}h_if_t^2(x_i)]} + \Omega(f_t)+constant$$

### 1.3 树的参数化

**树的参数化有两个：一个是对树模型的参数化，二是是对树的复杂度参数化。**

树模型参数化：如何定义一个树？主要定义两个部分：

1. 每棵树每个叶子节点的值（或者说每个叶子节点的权重）$w$：这是一个向量，因为每个树有很多叶子节点。
2. 样本到叶子节点的映射关系$q$（每个样本落在当前这个树的哪一个叶子节点上）

叶子节点样本归属集合$I$：每个叶子节点包含哪些样本。

树复杂度的参数化-如何定义树的复杂度：

1. 每棵树的叶子节点的个数（叶子节点个数越少模型越简单）
2. 叶子节点的权重值$w$（值越小模型越简单）

我们树的复杂度如下：

$$\Omega(f_t)=\gamma T+\frac{1}{2}\lambda \sum_{j=1}^T{w_j^2}$$

其中，$T$参数当前这棵叶子节点的个数；$w_j^2$是叶子节点值的$L_2$范数。

我们对树进行参数化后，带入到目标函数中：

$$Obj^{(t)}=\sum_{i=1}^n{[g_if_t(x_i)+\frac{1}{2}h_if_t^2(x_i)]} + \Omega(f_t)$$
$$=\sum_{i=1}^n{[g_iw_q(x_i)+\frac{1}{2}h_iw_q^2(x_i)]} + \gamma T + \frac{1}{2}\lambda \sum_{j=1}^T{w_j^2}$$
$$=\sum_{j=1}^T{[(\sum_{i\in I_j}g_i)w_j + \frac{1}{2}(\sum_{i\in I_j}+\lambda)w_j^2]} +\gamma T$$

最后一步的转化思路是从在这棵树中，每个样本落在哪个节点转为每个节点上有哪些样本。

记：

$$G_j=\sum_{i\in I_j}g_i$$
$$H_j=\sum_{i\in I_j}h_i$$

故：
$$Obj^{(t)}=\sum_{j=1}^T{[G_jw_j + \frac{1}{2}(H_j+\lambda)w_j^2]} +\gamma T$$

对$w_j$求偏导，可得：

$$w_j=-\frac{G_j}{H_j+\lambda}$$

所以目标函数可以化简为：

$$Obj = -\frac{1}{2}\sum_{j=1}^T{\frac{G_j^2}{H_j+\lambda}} + \gamma T$$

### 1.4 寻找树的形状-特征分裂

**分裂准则：基于当前特征分裂点前后，目标函数的差值**

#### 贪心算法：

1. 对每个叶节点枚举所有的可用特征；
2. 针对每个特征，把属于该节点的训练样本根据该特征值进行升序排列，通过线性扫描的方式来决定该特征的最佳分裂点，并记录该特征的分裂收益；
3. 选择收益最大的特征作为分裂特征，用该特征的最佳分裂点作为分裂位置，在该节点上分裂出左右两个新的叶节点，并4. 为每个新节点关联对应的样本集；
5. 回到第1步，递归执行直到满足特定条件为止

#### 近似算法-分位数候选点

对于每个特征，不去暴力搜索每个值，而是使用分位点

- 根据样本数量选择三分位点或者四分位点等等；
- 或者根据二阶导数（也就是梯度）作为权重进行划分

也就是说原来是某个特征的所有取值是候选点，现在是某个特征的分位点作为候选点。

## 2. 工程实现细节

### 2.0 特征分裂并行寻找

寻找特征分隔点需要对特征值进行排序，这个很耗时间。我们可以在训练之前先按照特征对样本数据进行排序，并分别保存在不同的块结构中。每个块都有一个按照某个特征排好序的特征加对应的一阶二阶导数。

对于不同的特征的特征划分点，XGBoost分别在不同的线程中并行选择分裂的最大增益

### 2.1 缓存访问

特征是排好序，但是通过索引获取一阶二阶导数值是不连续的，造成cpu缓存命中率低； xgboost解决办法：为每个线程分配一个连续的缓存区，将需要的梯度信息存放在缓冲区中，这样就连续了；

同时通过设置合理的分块的大小，充分利用了CPU缓存进行读取加速（cache-aware access）。使得数据读取的速度更快。另外，通过将分块进行压缩（block compressoin）并存储到硬盘上，并且通过将分块分区到多个硬盘上实现了更大的IO。

### 2.2 XGboost特征的重要性是如何评估的

- weight ：该特征在所有树中被用作分割样本的特征的总次数。
- gain ：该特征在其出现过的所有树中产生的平均增益（我自己的理解就是目标函数减少值总和的平均值，这里也可以使用增益之和）。
- cover ：该特征在其出现过的所有树中的平均覆盖范围。
这里有一个细节需要注意，就是节点分割的时候，之前用过的特征在当前节点也是可以使用的，所以weight才是有意义的。


## 3. 与GBDT相比，Xgboost的优化点：

- 算法本身的优化：首先GBDT只支持决策树，Xgboost除了支持决策树，可以支持多种弱学习器，可以是默认的gbtree, 也就是CART决策树，还可以是线性弱学习器gblinear以及DART；其次GBDT损失函数化简的时候进行的是一阶泰勒公式的展开，而Xgboost使用的是二阶泰勒公式的展示。还有一点是Xgboost的目标函数加上了正则项，这个正则项是对树复杂度的控制，防止过拟合。
- 可以处理缺失值。尝试通过枚举所有缺失值在当前节点是进入左子树，还是进入右子树更优来决定一个处理缺失值默认的方向
- 运行效率：并行化，单个弱学习器最耗时的就是决策树的分裂过程，对于不同特征的特征分裂点，可以使用多线程并行选择。这里想提一下，我自己理解，这里应该针对的是每个节点，而不是每个弱学习器。这里其实我当时深究了一下，有点混乱。为什么是针对每个节点呢？因为我每个节点也有很多特征，所以在每个节点上，我并行（多线程）除了多个特征，每个线程都在做寻找增益最大的分割点。还有需要注意的一点是Xgboost在并行处理之前，会提前把样本按照特征大小排好序，默认都放在右子树，然后递归的从小到大拿出一个个的样本放到左子树，然后计算对基于此时的分割点的增益的大小，然后记录并更新最大的增益分割点。

（1）GBDT是机器学习算法，XGBoost是该算法的工程实现。

（2）正则项：在使用CART作为基分类器时，XGBoost显式地加入了正则项来控制模型的复杂度，有利于防止过拟合，从而提高模型的泛化能力。

（3）导数信息：GBDT在模型训练时只使用了代价函数的一阶导数信息，XGBoost对代价函数进行二阶泰勒展开，可以同时使用一阶和二阶导数。

（4）基分类器：传统的GBDT采用CART作为基分类器，XGBoost支持多种类型的基分类器，比如线性分类器。

（5）子采样：传统的GBDT在每轮迭代时使用全部的数据，XGBoost则采用了与随机森林相似的策略，支持对数据进行采样。

（6）缺失值处理：传统GBDT没有设计对缺失值进行处理，XGBoost能够自动学习出缺失值的处理策略。

（7）并行化：传统GBDT没有进行并行化设计，注意不是tree维度的并行，而是特征维度的并行。XGBoost预先将每个特征按特征值排好序，存储为块结构，分裂结点时可以采用多线程并行查找每个特征的最佳分割点，极大提升训练速度。

## 4. XGBoost的优缺点
### 4.1 优点

- 精度更高：GBDT 只用到一阶泰勒展开，而 XGBoost 对损失函数进行了二阶泰勒展开。XGBoost 引入二阶导一方面是为了增加精度，另一方面也是为了能够自定义损失函数，二阶泰勒展开可以近似大量损失函数；
- 灵活性更强：GBDT 以 CART 作为基分类器，XGBoost 不仅支持 CART 还支持线性分类器，使用线性分类器的 XGBoost 相当于带 L1 和 L2 正则化项的逻辑斯蒂回归（分类问题）或者线性回归（回归问题）。此外，XGBoost 工具支持自定义损失函数，只需函数支持一阶和二阶求导；
- 正则化：XGBoost 在目标函数中加入了正则项，用于控制模型的复杂度。正则项里包含了树的叶子节点个数、叶子节点权重的 L2 范式。正则项降低了模型的方差，使学习出来的模型更加简单，有助于防止过拟合，这也是XGBoost优于传统GBDT的一个特性。
- Shrinkage（缩减）：相当于学习速率。XGBoost 在进行完一次迭代后，会将叶子节点的权重乘上该系数，主要是为了削弱每棵树的影响，让后面有更大的学习空间。传统GBDT的实现也有学习速率；
- 列抽样：XGBoost 借鉴了随机森林的做法，支持列抽样，不仅能降低过拟合，还能减少计算。这也是XGBoost异于传统GBDT的一个特性；
缺失值处理：对于特征的值有缺失的样本，XGBoost 采用的稀疏感知算法可以自动学习出它的分裂方向；
- XGBoost工具支持并行：boosting不是一种串行的结构吗?怎么并行的？注意XGBoost的并行不是tree粒度的并行，XGBoost也是一次迭代完才能进行下一次迭代的（第t次迭代的代价函数里包含了前面t-1次迭代的预测值）。XGBoost的并行是在特征粒度上的。我们知道，决策树的学习最耗时的一个步骤就是对特征的值进行排序（因为要确定最佳分割点），XGBoost在训练之前，预先对数据进行了排序，然后保存为block结构，后面的迭代中重复地使用这个结构，大大减小计算量。这个block结构也使得并行成为了可能，在进行节点的分裂时，需要计算每个特征的增益，最终选增益最大的那个特征去做分裂，那么各个特征的增益计算就可以开多线程进行。
- 可并行的近似算法：树节点在进行分裂时，我们需要计算每个特征的每个分割点对应的增益，即用贪心法枚举所有可能的分割点。当数据无法一次载入内存或者在分布式情况下，贪心算法效率就会变得很低，所以XGBoost还提出了一种可并行的近似算法，用于高效地生成候选的分割点。

### 4.2 缺点

- 虽然利用预排序和近似算法可以降低寻找最佳分裂点的计算量，但在节点分裂过程中仍需要遍历数据集；
- 预排序过程的空间复杂度过高，不仅需要存储特征值，还需要存储特征对应样本的梯度统计值的索引，相当于消耗了两倍的内存。

https://zhuanlan.zhihu.com/p/83901304

面试题：

https://mp.weixin.qq.com/s/RSQWx4fH3uI_sjZzAKVyKQ

https://mp.weixin.qq.com/s/BbelOsYgsiOvwfwYs5QfpQ

https://mp.weixin.qq.com/s/_QgnYoW827GDgVH9lexkNA

